# Weighted K Nearest Neighbors

Algorithm: compute the K nearest neighbors of a trajectory as the predicted trajectories

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import multiprocessing as mp
from tqdm import tnrange, tqdm_notebook, tqdm
import time
import os
import sys

In [2]:
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
from argoverse.map_representation.map_api import ArgoverseMap
from argoverse.visualization.visualize_sequences import viz_sequence
from IPython.display import clear_output

In [3]:
def normalize_trajectory(traj):
    norm_traj = traj - traj[0]
    theta = -np.arctan2(norm_traj[19][1],norm_traj[19][0])
    rot_mat = ((np.cos(theta), -np.sin(theta)), (np.sin(theta), np.cos(theta)))
    norm_traj = np.matmul(rot_mat, norm_traj.T).T
    return norm_traj

In [4]:
def build_lookup(afl, output_path):
    if os.path.exists(output_path):
        sys.exit(f'output path \"{output_path}\" already exists')
    num_seqs = len(afl)
    lookup = [None] * num_seqs
    #print(f'Processing {num_seqs} sequences')
    for idx in tqdm(range(num_seqs)):
        seq = afl[idx]
        lookup[idx] = normalize_trajectory(seq.agent_traj)
    lookup = np.asarray(lookup)
    np.save(output_path, lookup)
        

In [ ]:
root_dir = '/home/aaron/workspace/argoverse/data/forecasting_train_v1.1/train/data'
afl = ArgoverseForecastingLoader(root_dir)
build_lookup(afl, 'lookup.npy')

In [ ]:
root_dir = '/home/aaron/workspace/argoverse/data/forecasting_val_v1.1/val/data'
afl = ArgoverseForecastingLoader(root_dir)
val_trajectories = [None] * len(afl)
for idx in tqdm(range(len(afl))):
    val_trajectories[idx] = afl[idx].agent_traj
np.save('val_trajectories.npy',val_trajectories)

In [5]:
def get_top_k(target_traj, lookup, k=6):
    norm_target_traj = normalize_trajectory(target_traj)
    distances = np.subtract(lookup[:,:20], norm_target_traj[:20])
    distances = np.linalg.norm(distances, axis=(1,2))
    top_k_idxs = np.argsort(distances)
    return distances, top_k_idxs[:k]

In [6]:
def smooth_prediction(predict_traj, agent_traj):
    agent_dx = agent_traj[19,0] - agent_traj[18,0]
    agent_dy = agent_traj[19,1] - agent_traj[18,1]
    target_x = agent_traj[19,0] + agent_dx
    target_y = agent_traj[19,1] + agent_dy
    shift_x = target_x - predict_traj[20,0]
    shift_y = target_y - predict_traj[20,1]
    predict_traj[:,0] += shift_x
    predict_traj[:,1] += shift_y
    return predict_traj

In [7]:
def compute_metrics(predictions, agent_traj):
    #target_traj = np.asarray(test_seq.agent_traj)
    distances = predictions-agent_traj
    error = np.linalg.norm(distances[:,20:,:],axis=2)
    final_error = error[:,-1]
    fde_idx = np.argmin(final_error)
    fde = final_error[fde_idx]
    ade = np.mean(error[fde_idx])
    misses = len(final_error[final_error > 2.0])
    mr = misses/predictions.shape[0]
    return fde_idx, fde, ade, mr

In [8]:
def get_multiple_forecasts(agent_traj, lookup, k=6, plot=False, is_test=True):
    predictions = [None] * k
    #agent_traj = test_seq.agent_traj
    origin_agent_traj = agent_traj - agent_traj[0]
    norm_agent_traj = normalize_trajectory(agent_traj)
    distances, top_k_idxs = get_top_k(norm_agent_traj, lookup, k=6)
    for idx, k in enumerate(top_k_idxs):
        predict_traj = lookup[k]
        # using cosine dot product relation to determine angular difference between trajectories
        arg = np.dot(predict_traj[19], origin_agent_traj[19])/ np.linalg.norm(predict_traj[19])/np.linalg.norm(origin_agent_traj[19])
        theta = np.arccos(np.clip(arg, -1.0, 1.0))
        if origin_agent_traj[19,1] < 0:
            # rotate in negative angle direction (cw)
            theta *= -1
        rot_mat = ((np.cos(theta), -np.sin(theta)), (np.sin(theta), np.cos(theta)))
        t_predict_traj = np.matmul(rot_mat, predict_traj.T).T + agent_traj[0]
#         if np.linalg.norm(t_predict_traj[20]-agent_traj[19]) > 1:
#             t_predict_traj = smooth_prediction(t_predict_traj, agent_traj)
        t_predict_traj[:20] = agent_traj[:20]
        predictions[idx] = t_predict_traj
    predictions = np.asarray(predictions)
    
    #metrics = compute_metrics(predictions, test_seq)
    if not is_test:
        metrics = compute_metrics(predictions, agent_traj)
        fde_idx, fde, ade, mr = metrics
    else:
        metrics = None
    if plot:
        boldwidth=3
        fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10,10))
        #ax = plt.gca().set_aspect('equal')
        # plot starting point
        ax[0].set_title('predictions')
        ax[0].plot(agent_traj[0,0], agent_traj[0,1],'-o', c='r')
        ax[0].plot(agent_traj[:,0], agent_traj[:,1],c='r',linewidth=boldwidth)
        for idx, prediction in enumerate(predictions):
            if not is_test:
                linewidth = boldwidth if idx == fde_idx else 1
                linecolor = 'b' if idx == fde_idx else np.random.random(3,)
            else:
                linewidth = 1
                linecolor = np.random.random(3,)
                
            
            #ax[0].plot(prediction[19,0], prediction[19,1],'-o',c=linecolor)
            ax[0].plot(prediction[19:,0], prediction[19:,1],c=linecolor, linewidth=linewidth)
        ax[1].set_title('normalized predictions')
        ax[1].plot(norm_agent_traj[0,0], norm_agent_traj[0,1],'-o', c='r')
        ax[1].plot(norm_agent_traj[:,0], norm_agent_traj[:,1],c='r',linewidth=boldwidth)
        for idx, k in enumerate(top_k_idxs):
            if not is_test:
                linewidth = boldwidth if idx == fde_idx else 1
                linecolor = 'b' if idx == fde_idx else np.random.random(3,)
            else:
                linewidth = 1
                linecolor = np.random.random(3,)
            norm_prediction = lookup[k]
            #ax[1].plot(norm_prediction[19,0], norm_prediction[19,1],'-o',c=linecolor)
            ax[1].plot(norm_prediction[19:,0], norm_prediction[19:,1],c=linecolor, linewidth=linewidth)
    return top_k_idxs, predictions, metrics
    

In [ ]:
root_dir = '/home/aaron/workspace/argoverse/data/forecasting_test_v1.1/test_obs/data'
#root_dir = '/home/aaron/workspace/argoverse/data/forecasting_sample_v1.1/forecasting_sample/data'
afl = ArgoverseForecastingLoader(root_dir)
lookup = np.load('lookup.npy', allow_pickle=True)


In [ ]:
idx = np.random.randint(len(afl))
agent_traj = afl[idx].agent_traj
top_k_idxs, predictions, metrics = get_multiple_forecasts(agent_traj, lookup, plot=True, is_test=True)
print(predictions.shape)

In [11]:
output_all = dict()
root_dir = '/home/aaron/workspace/argoverse/data/forecasting_test_v1.1/test_obs/data'
#root_dir = '/home/aaron/workspace/argoverse/data/forecasting_sample_v1.1/forecasting_sample/data'
afl = ArgoverseForecastingLoader(root_dir)
test_trajectories = np.load('test_trajectories.npy')
lookup = np.load('lookup.npy')
for idx in tqdm(range(len(test_trajectories))):
#for idx in tqdm(range(10)):
    agent_traj = test_trajectories[idx]
    data = afl[idx]
    top_k_idxs, predictions, metrics = get_multiple_forecasts(agent_traj, lookup)
    seq_id = int(data.current_seq.name[:-4])
    output_all[seq_id] = predictions[:,20:,:]
np.save('knn_baseline_output_all.npy', output_all)
# times = list()
# for traj in test_trajectories[:10]:
#     start = time.time()
#     get_multiple_forecasts(traj, lookup)
#     times.append(time.time()-start)
# print(np.mean(times))

100%|██████████| 78143/78143 [1:44:51<00:00, 12.42it/s]


In [12]:
from argoverse.evaluation.competition_util import generate_forecasting_h5

output_path = 'competition_files/'

generate_forecasting_h5(output_all, output_path, filename='knn_baseline') #this might take awhile

78143/78143

In [1]:
len(output_all.keys())

NameError: name 'output_all' is not defined